In [1]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emeld\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data_path = "../data/txt/"

# Choix d'une décennie et du nombre de clusters

In [4]:
DECADE = '1920'
N_CLUSTERS = 10

# Chargement des fichiers de la décennie

In [5]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [6]:
# Exemple de fichiers
files [:2]

['Bxl_1920_Tome_I2_Part_1.txt', 'Bxl_1920_Tome_I2_Part_10.txt']

In [7]:
texts = [open(data_path + f, encoding='utf8', errors='ignore').read() for f in files]

In [8]:
# Exemple de textes
texts[0][:400]

'\x0cVILLE\n\nDÈ\n\nBRUXELLES\n\nBULLETIN COMMUNAL\nA N N É E\n\nPREMIÈRE\nTOME\n\nCOMPTE\n\nRENDU\n\n1920\n\nPARTIE\nII\n\nDES\n\nSÉANCES\n\nBRUXELLES\nTYPOGRAPHIE ET LITHOGRAPHIE E. GUYOT\n12, rue P a c l i é c o ,\n\n1920\n\n12\n\n\x0c\x0cCOMPTE RENDU DE L A SÉANCE D U 1 2 J U I L L E T 1 9 2 0 .\n\nVILLE\n\nD E\n\nBULLETIN\n\nCOMMUNAL\n\nAnnée\n\nC O N S E I L\nSéance\n\nB R U X E L L E S\n\n1920.\n\nC O M M U N A L\ndu 1 2 Juillet\n\n1920.\n\nPrésidence de M'

# Vectorisation du texte

In [9]:
def process_text(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [10]:
vectorizer = TfidfVectorizer(tokenizer=process_text,
                                stop_words=stopwords.words('french'),
                                max_df=0.5,
                                min_df=0.1,
                                lowercase=True)

In [11]:
%time tfidf_vectors = vectorizer.fit_transform(texts)

Wall time: 57.4 s


In [12]:
tfidf_vectors

<178x6665 sparse matrix of type '<class 'numpy.float64'>'
	with 251545 stored elements in Compressed Sparse Row format>

In [13]:
# Exemple de vecteur TFIDF
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names()
    ).sort_values(ascending=False)

camille       0.236540
huysmans      0.231977
conrardy      0.221826
hallet        0.188366
bremaecker    0.153632
                ...   
literies      0.000000
litre         0.000000
litres        0.000000
lits          0.000000
#             0.000000
Length: 6665, dtype: float64

# Comprendre les vecteurs et leurs "distances"

In [14]:
from scipy.spatial.distance import cosine

In [15]:
cosine([1, 2, 3], [1, 2, 3])

0.0

In [16]:
cosine([1, 2, 3], [1, 2, 2])

0.02004211298777725

In [17]:
cosine([1, 2, 3], [2, 2, 2])

0.07417990022744858

In [18]:
tfidf_array = tfidf_vectors.toarray()

In [19]:
tfidf_array[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [20]:
texts[0][:200]

'\x0cVILLE\n\nDÈ\n\nBRUXELLES\n\nBULLETIN COMMUNAL\nA N N É E\n\nPREMIÈRE\nTOME\n\nCOMPTE\n\nRENDU\n\n1920\n\nPARTIE\nII\n\nDES\n\nSÉANCES\n\nBRUXELLES\nTYPOGRAPHIE ET LITHOGRAPHIE E. GUYOT\n12, rue P a c l i é c o ,\n\n1920\n\n12\n\n\x0c\x0cC'

In [21]:
tfidf_array[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [22]:
cosine(tfidf_array[0], tfidf_array[1])

0.8879884802924477

# Clustering des vecteurs TFIDF

Article intéressant sur le KMeans clustering :
- https://medium.com/dataseries/k-means-clustering-explained-visually-in-5-minutes-b900cc69d175

In [23]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [24]:
km_model.fit(tfidf_vectors)

KMeans(n_clusters=10)

In [25]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(km_model.labels_):
    clustering[label].append(files[idx])

In [26]:
pprint(dict(clustering))

{0: ['Bxl_1920_Tome_I2_Part_10.txt',
     'Bxl_1920_Tome_I2_Part_11.txt',
     'Bxl_1920_Tome_I2_Part_8.txt',
     'Bxl_1920_Tome_I2_Part_9.txt',
     'Bxl_1925_Tome_II1_2_Part_3.txt',
     'Bxl_1925_Tome_II1_2_Part_4.txt',
     'Bxl_1925_Tome_II1_2_Part_5.txt',
     'Bxl_1925_Tome_II1_2_Part_6.txt',
     'Bxl_1926_Tome_I2_Part_12.txt',
     'Bxl_1926_Tome_I2_Part_13.txt',
     'Bxl_1926_Tome_I2_Part_14.txt',
     'Bxl_1926_Tome_II1_Part_12.txt',
     'Bxl_1926_Tome_II1_Part_13.txt',
     'Bxl_1926_Tome_II1_Part_14.txt',
     'Bxl_1928_Tome_II1_Part_10.txt',
     'Bxl_1928_Tome_II1_Part_11.txt',
     'Bxl_1928_Tome_II1_Part_9.txt'],
 1: ['Bxl_1921_Tome_I1_Part_8.txt',
     'Bxl_1923_Tome_I1_Part_1.txt',
     'Bxl_1923_Tome_I1_Part_2.txt',
     'Bxl_1923_Tome_I1_Part_3.txt',
     'Bxl_1923_Tome_I1_Part_4.txt',
     'Bxl_1923_Tome_I1_Part_5.txt',
     'Bxl_1923_Tome_I1_Part_6.txt',
     'Bxl_1923_Tome_I1_Part_8.txt',
     'Bxl_1923_Tome_I2_Part_1.txt',
     'Bxl_1923_Tome_I2_Part_4.txt',

In [31]:
import os
import yake

ignored = set(["conseil communal", "conseil général"])
kw_extractor = yake.KeywordExtractor(lan="fr", top=20)

data_path = "../data/txt/"
files = os.listdir(data_path)

In [32]:
cluster_0 = ['Bxl_1920_Tome_I2_Part_10.txt',
     'Bxl_1920_Tome_I2_Part_11.txt',
     'Bxl_1920_Tome_I2_Part_9.txt',
     'Bxl_1925_Tome_II1_2_Part_3.txt',
     'Bxl_1925_Tome_II1_2_Part_4.txt',
     'Bxl_1925_Tome_II1_2_Part_5.txt',
     'Bxl_1925_Tome_II1_2_Part_6.txt',
     'Bxl_1926_Tome_I2_Part_12.txt',
     'Bxl_1926_Tome_I2_Part_13.txt',
     'Bxl_1926_Tome_I2_Part_14.txt',
     'Bxl_1926_Tome_II1_Part_12.txt',
     'Bxl_1926_Tome_II1_Part_13.txt',
     'Bxl_1926_Tome_II1_Part_14.txt',
     'Bxl_1928_Tome_II1_Part_10.txt',
     'Bxl_1928_Tome_II1_Part_9.txt']

In [33]:
for f in sorted(cluster_0):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1920_Tome_I2_Part_10.txt mentions these keywords: l'année scolaire, l'année scolaire bruxelles, cours professionnels, nombre d'élèves, population moyenne, commission administrative, nombre d'élèves habitant, comité administratif, école professionnelle...
Bxl_1920_Tome_I2_Part_11.txt mentions these keywords: comité central industriel, comité central, délégué du comité, chambre syndicale, central industriel, ville de bruxelles, conseiller communal, ministre plénipotentiaire, comité national, envoyé extraordinaire, comité exécutif...
Bxl_1920_Tome_I2_Part_9.txt mentions these keywords: conseiller communal, cours d'éducation, membres délégués, cours cours, écoles population membres, cours cours inférieurs, école moyenne...
Bxl_1925_Tome_II1_2_Part_3.txt mentions these keywords: école école école, jardin d'enfants, l'année scolaire, ecole communale, avenue louise, écoles primaires communales, écoles primaires...
Bxl_1925_Tome_II1_2_Part_4.txt mentions these keywords: jeunes filles, memb

In [34]:
cluster_1 = ['Bxl_1921_Tome_I1_Part_8.txt',
     'Bxl_1923_Tome_I1_Part_1.txt',
     'Bxl_1923_Tome_I1_Part_2.txt',
     'Bxl_1923_Tome_I1_Part_3.txt',
     'Bxl_1923_Tome_I1_Part_4.txt',
     'Bxl_1923_Tome_I1_Part_5.txt',
     'Bxl_1923_Tome_I1_Part_6.txt',
     'Bxl_1923_Tome_I1_Part_8.txt',
     'Bxl_1923_Tome_I2_Part_1.txt',
     'Bxl_1923_Tome_I2_Part_4.txt',
     'Bxl_1924_Tome_I1_Part_1.txt',
     'Bxl_1924_Tome_I1_Part_2.txt',
     'Bxl_1924_Tome_I1_Part_3.txt',
     'Bxl_1924_Tome_I1_Part_4.txt',
     'Bxl_1925_Tome_II1_1_Part_1.txt',
     'Bxl_1925_Tome_II1_1_Part_5.txt',
     'Bxl_1926_Tome_I1_Part_1.txt',
     'Bxl_1926_Tome_I1_Part_2.txt',
     'Bxl_1926_Tome_I1_Part_3.txt',
     'Bxl_1926_Tome_I1_Part_4.txt',
     'Bxl_1926_Tome_I1_Part_5.txt',
     'Bxl_1926_Tome_I1_Part_6.txt',
     'Bxl_1926_Tome_I2_Part_1.txt',
     'Bxl_1926_Tome_I2_Part_2.txt',
     'Bxl_1926_Tome_I2_Part_7.txt',
     'Bxl_1926_Tome_II1_Part_1.txt',
     'Bxl_1926_Tome_II1_Part_7.txt',
     'Bxl_1927_Tome_I_Part_6.txt']

In [35]:
for f in sorted(cluster_1):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1921_Tome_I1_Part_8.txt mentions these keywords: verhaeghe de naeyer, conseil général d'administration, ville de bruxelles, van den nest...
Bxl_1923_Tome_I1_Part_1.txt mentions these keywords: mesdames et messieurs, crédit supplémentaire, ville de bruxelles, avis favorable, collège a l'honneur...
Bxl_1923_Tome_I1_Part_2.txt mentions these keywords: crédit supplémentaire, avis favorable, mesdames et messieurs, écoles moyennes...
Bxl_1923_Tome_I1_Part_3.txt mentions these keywords: mesdames et messieurs, personnel occupé, personnel enseignant...
Bxl_1923_Tome_I1_Part_4.txt mentions these keywords: conseil d'appel, conseil de prud'hommes...
Bxl_1923_Tome_I1_Part_5.txt mentions these keywords: conseil d'appel, ville de bruxelles, mesdames et messieurs, hospice hospice refuge, conseil de prud'hommes, conseil général d'administration, crédit supplémentaire...
Bxl_1923_Tome_I1_Part_6.txt mentions these keywords: mesdames et messieurs, crédit supplémentaire, conseil des hospices, avis favo

In [40]:
cluster_2 = ['Bxl_1920_Tome_I2_Part_5.txt',
     'Bxl_1923_Tome_I2_Part_5.txt',
     'Bxl_1925_Tome_II1_1_Part_6.txt',
     'Bxl_1926_Tome_I2_Part_8.txt',
     'Bxl_1926_Tome_II1_Part_8.txt',
     'Bxl_1928_Tome_II1_Part_5.txt',
     'Bxl_1928_Tome_II2_Part_10.txt',
     'Bxl_1928_Tome_II2_Part_11.txt',
     'Bxl_1928_Tome_II2_Part_12.txt',
     'Bxl_1929_Tome_II1_Part_5.txt',
     'Bxl_1929_Tome_II1_Part_6.txt']

In [37]:
for f in sorted(cluster_2):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1920_Tome_I2_Part_5.txt mentions these keywords: votes de liste, listes électorales, liste attribués, votes nominatifs...
Bxl_1923_Tome_I2_Part_5.txt mentions these keywords: don de l'auteur...
Bxl_1925_Tome_II1_1_Part_6.txt mentions these keywords: nombre des votes, nombre nombre montant, listes électorales, nombre total, votes de liste, liquidation nombre nombre, nombre de sièges, nombre nombre...
Bxl_1926_Tome_I2_Part_8.txt mentions these keywords: nombre des votes, division nombre, district district, liste total chiffre, votes de liste...
Bxl_1926_Tome_II1_Part_8.txt mentions these keywords: division nombre, permis de conduire, nombre des votes...
Bxl_1928_Tome_II1_Part_5.txt mentions these keywords: nombre de sièges, liste attribués, sièges attribués, liste nombre, nombre total, nombre de votes, nombre de suffrages, votes nominatifs, district district total, candidats nombre, liste votes nominatifs...
Bxl_1928_Tome_II2_Part_10.txt mentions these keywords: maison rue, maison ru

In [41]:
cluster_3 = ['Bxl_1921_Tome_I1_Part_6.txt',
     'Bxl_1923_Tome_I1_Part_9.txt',
     'Bxl_1923_Tome_I2_Part_2.txt',
     'Bxl_1924_Tome_I1_Part_5.txt',
     'Bxl_1924_Tome_I1_Part_6.txt',
     'Bxl_1924_Tome_I1_Part_7.txt',
     'Bxl_1926_Tome_I1_Part_9.txt',
     'Bxl_1926_Tome_I2_Part_3.txt',
     'Bxl_1926_Tome_II1_Part_3.txt',
     'Bxl_1927_Tome_I_Part_12.txt',
     'Bxl_1927_Tome_I_Part_13.txt',
     'Bxl_1927_Tome_I_Part_9.txt',
     'Bxl_1928_Tome_II1_Part_12.txt',
     'Bxl_1929_Tome_I_Part_4.txt']

In [39]:
for f in sorted(cluster_3):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1921_Tome_I1_Part_6.txt mentions these keywords: frais d'entretien, nature budget budget, montant des dépenses, l'exercice conseil général, dépenses portées, service médical, service administratif, dépenses présumées, conseil général d'administration, service intérieur...
Bxl_1923_Tome_I1_Part_9.txt mentions these keywords: écoles primaires...
Bxl_1923_Tome_I2_Part_2.txt mentions these keywords: conseil général collège, l'exercice conseil général, l'exercice conseil, général collège, frais service administratif, frais frais, traitement du personnel, service administratif, frais d'entretien, service frais intérieur...
Bxl_1924_Tome_I1_Part_5.txt mentions these keywords: conseil des hospices, frais d'entretien, van de meulebroeck...
Bxl_1924_Tome_I1_Part_6.txt mentions these keywords: conseil général conseil, l'exercice conseil général, général conseil communal, frais d'entretien, service administratif, service médical, montant des dépenses, traitement du personnel...
Bxl_1924_Tome_I

In [42]:
cluster_4 = ['Bxl_1923_Tome_I2_Part_9.txt',
     'Bxl_1926_Tome_I1_Part_7.txt',
     'Bxl_1926_Tome_II1_Part_2.txt',
     'Bxl_1927_Tome_I_Part_1.txt',
     'Bxl_1927_Tome_I_Part_2.txt',
     'Bxl_1927_Tome_I_Part_3.txt',
     'Bxl_1927_Tome_I_Part_4.txt',
     'Bxl_1927_Tome_I_Part_5.txt',
     'Bxl_1927_Tome_I_Part_7.txt',
     'Bxl_1927_Tome_I_Part_8.txt',
     'Bxl_1928_Tome_II1_Part_1.txt',
     'Bxl_1928_Tome_II1_Part_3.txt',
     'Bxl_1928_Tome_II1_Part_4.txt',
     'Bxl_1928_Tome_II2_Part_1.txt',
     'Bxl_1928_Tome_II2_Part_2.txt',
     'Bxl_1928_Tome_II2_Part_3.txt',
     'Bxl_1928_Tome_II2_Part_4.txt',
     'Bxl_1928_Tome_II2_Part_5.txt',
     'Bxl_1928_Tome_II2_Part_6.txt',
     'Bxl_1929_Tome_II1_Part_1.txt',
     'Bxl_1929_Tome_II1_Part_2.txt',
     'Bxl_1929_Tome_II2_Part_1.txt',
     'Bxl_1929_Tome_II2_Part_2.txt',
     'Bxl_1929_Tome_II2_Part_3.txt',
     'Bxl_1929_Tome_II2_Part_4.txt',
     'Bxl_1929_Tome_II2_Part_5.txt',
     'Bxl_1929_Tome_II2_Part_6.txt',
     'Bxl_1929_Tome_II2_Part_7.txt',
     'Bxl_1929_Tome_I_Part_1.txt',
     'Bxl_1929_Tome_I_Part_2.txt',
     'Bxl_1929_Tome_I_Part_3.txt',
     'Bxl_1929_Tome_I_Part_7.txt',
     'Bxl_1929_Tome_I_Part_8.txt',
     'Bxl_1929_Tome_I_Part_9.txt']

In [43]:
for f in sorted(cluster_4):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1923_Tome_I2_Part_9.txt mentions these keywords: l'ordre public, loi communale...
Bxl_1926_Tome_I1_Part_7.txt mentions these keywords: stationnement stationnement...
Bxl_1926_Tome_II1_Part_2.txt mentions these keywords: l'echevin jacqmain, société bruxelloise, van remoortel, pouvoir autorisant...
Bxl_1927_Tome_I_Part_1.txt mentions these keywords: van overstraeten, van remoortel, van den nest, huisman van den, monsieur van overstraeten, huisman van, mesdames et messieurs, van den, van overstraeten s'est...
Bxl_1927_Tome_I_Part_2.txt mentions these keywords: van overstraeten, mesdames et messieurs, ville de bruxelles, l'echevin lemonnier, tramways bruxellois, l'echevin coelst, huisman van den...
Bxl_1927_Tome_I_Part_3.txt mentions these keywords: van remoortel, l'echevin coelst, ville de bruxelles, van overstraeten, écoles libres, l'echevin pattou, l'echevin jacqmain, l'enseignement libre...
Bxl_1927_Tome_I_Part_4.txt mentions these keywords: l'echevin wauwermans, van overstraeten, 

In [46]:
cluster_5 = ['Bxl_1920_Tome_I2_Part_1.txt',
     'Bxl_1920_Tome_I2_Part_12.txt',
     'Bxl_1920_Tome_I2_Part_13.txt',
     'Bxl_1920_Tome_I2_Part_14.txt',
     'Bxl_1920_Tome_I2_Part_18.txt',
     'Bxl_1920_Tome_I2_Part_19.txt',
     'Bxl_1920_Tome_I2_Part_3.txt',
     'Bxl_1920_Tome_I2_Part_4.txt',
     'Bxl_1921_Tome_I1_Part_1.txt',
     'Bxl_1921_Tome_I1_Part_2.txt',
     'Bxl_1921_Tome_I1_Part_3.txt',
     'Bxl_1921_Tome_I1_Part_4.txt',
     'Bxl_1921_Tome_I1_Part_5.txt',
     'Bxl_1921_Tome_I1_Part_7.txt',
     'Bxl_1921_Tome_I1_Part_9.txt',
     'Bxl_1922_Tome_II1_Part_4.txt',
     'Bxl_1923_Tome_I1_Part_10.txt',
     'Bxl_1923_Tome_I1_Part_7.txt',
     'Bxl_1925_Tome_II1_1_Part_2.txt',
     'Bxl_1925_Tome_II1_2_Part_7.txt',
     'Bxl_1925_Tome_II1_2_Part_8.txt',
     'Bxl_1926_Tome_I1_Part_8.txt',
     'Bxl_1926_Tome_I2_Part_15.txt',
     'Bxl_1926_Tome_II1_Part_15.txt',
     'Bxl_1929_Tome_I_Part_10.txt']

In [57]:
for f in sorted(cluster_5):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1920_Tome_I2_Part_1.txt mentions these keywords: camille huysmans, section des finances, avis favorable, l'echevin hallet...
Bxl_1920_Tome_I2_Part_12.txt mentions these keywords: paiement d'une somme, maison rue...
Bxl_1920_Tome_I2_Part_13.txt mentions these keywords: section des finances, avis favorable, collège vous propose, crédit supplémentaire, finances a émis, vote d'un crédit, ressources ordinaires, dépenses ordinaires...
Bxl_1920_Tome_I2_Part_14.txt mentions these keywords: l'echevin lemonnier, allocations allocations dépenses, francs francs francs, l'echevin jacqmain, allocations allocations, allocations allocations modifiées...
Bxl_1920_Tome_I2_Part_18.txt mentions these keywords: maison rue, supplément jusqu'à nouvel, jusqu'à nouvel ordre, supplément jusqu'à, nouvel ordre, jusqu'à nouvel...
Bxl_1920_Tome_I2_Part_19.txt mentions these keywords: titre définitif, nommée institutrice primaire, concession de terrain, comité scolaire, terrain pour sépulture, primaire à titre, 

UnboundLocalError: local variable 'block_of_word_obj' referenced before assignment

In [50]:
cluster_6 = ['Bxl_1920_Tome_I2_Part_6.txt',
     'Bxl_1923_Tome_I2_Part_6.txt',
     'Bxl_1925_Tome_II1_1_Part_7.txt',
     'Bxl_1926_Tome_I2_Part_9.txt',
     'Bxl_1926_Tome_II1_Part_9.txt',
     'Bxl_1928_Tome_II1_Part_6.txt',
     'Bxl_1929_Tome_II1_Part_7.txt']

In [49]:
for f in sorted(cluster_6):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1920_Tome_I2_Part_6.txt mentions these keywords: divisions territoriales, sexe masculin, classe pauvre...
Bxl_1923_Tome_I2_Part_6.txt mentions these keywords: naissances vivantes...
Bxl_1925_Tome_II1_1_Part_7.txt mentions these keywords: naissances vivantes, naissances vivantes enregistrées, parents habitant bruxelles, maladies contagieuses...
Bxl_1926_Tome_I2_Part_9.txt mentions these keywords: naissances vivantes, enfants légitimes, parents habitant bruxelles...
Bxl_1926_Tome_II1_Part_9.txt mentions these keywords: sexe masc, l'hygiène publique...
Bxl_1928_Tome_II1_Part_6.txt mentions these keywords: liste votes nominatifs, nombre de sièges, sièges attribués, liste votes, nombre de votes, votes nominatifs, nombre total, sexe masc...
Bxl_1929_Tome_II1_Part_7.txt mentions these keywords: mortalité due, conséquences de l'accouchement, tuberculose pulmonaire, causes décès, naissances enregistrées, diarrhée et entérite...


In [51]:
cluster_7 = ['Bxl_1920_Tome_I2_Part_7.txt',
     'Bxl_1922_Tome_II1_Part_1.txt',
     'Bxl_1922_Tome_II1_Part_2.txt',
     'Bxl_1922_Tome_II1_Part_3.txt',
     'Bxl_1923_Tome_I2_Part_7.txt',
     'Bxl_1923_Tome_I2_Part_8.txt',
     'Bxl_1923_Tome_II1_Part_1.txt',
     'Bxl_1923_Tome_II1_Part_2.txt',
     'Bxl_1923_Tome_II1_Part_3.txt',
     'Bxl_1925_Tome_II1_2_Part_1.txt',
     'Bxl_1925_Tome_II1_2_Part_2.txt',
     'Bxl_1926_Tome_I2_Part_10.txt',
     'Bxl_1926_Tome_I2_Part_11.txt',
     'Bxl_1926_Tome_II1_Part_10.txt',
     'Bxl_1926_Tome_II1_Part_11.txt',
     'Bxl_1928_Tome_II1_Part_7.txt',
     'Bxl_1928_Tome_II1_Part_8.txt']

In [52]:
for f in sorted(cluster_7):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1920_Tome_I2_Part_7.txt mentions these keywords: voie publique, décès voie publique, service s'est occupé, ans. ans. ans., service s'est, population décès voie...
Bxl_1922_Tome_II1_Part_1.txt mentions these keywords: collège des bourgmestre, l'hôtel de ville, bourgmestre et échevins, cahier des charges, ville jusqu'au jour, voies publiques...
Bxl_1922_Tome_II1_Part_2.txt mentions these keywords: école école école, rue rue rue, collège des bourgmestre, cahier des charges, école école, l'hôtel de ville, société royale, bourgmestre et echevins...
Bxl_1922_Tome_II1_Part_3.txt mentions these keywords: voie publique, collège des bourgmestre, bourgmestre et echevins, l'hôtel de ville, cahier des charges...
Bxl_1923_Tome_I2_Part_7.txt mentions these keywords: voie publique, divisions territoriales, décès voie publique, voie décès publique, bronchite aiguë, district divisions territoriales, méningite aiguë...
Bxl_1923_Tome_I2_Part_8.txt mentions these keywords: ...
Bxl_1923_Tome_II1_Part_1.

In [53]:
cluster_8 = ['Bxl_1926_Tome_I2_Part_4.txt',
     'Bxl_1926_Tome_I2_Part_5.txt',
     'Bxl_1926_Tome_II1_Part_4.txt',
     'Bxl_1926_Tome_II1_Part_5.txt',
     'Bxl_1927_Tome_I_Part_10.txt',
     'Bxl_1927_Tome_I_Part_11.txt',
     'Bxl_1929_Tome_I_Part_5.txt',
     'Bxl_1929_Tome_I_Part_6.txt']

In [54]:
for f in sorted(cluster_8):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1926_Tome_I2_Part_4.txt mentions these keywords: obligation compagnie belge, obligation compagnie, anciennement art, chemin de fer, obligations tramways, obligations chemin, compagnie belge chemins, francs le kilogramme...
Bxl_1926_Tome_I2_Part_5.txt mentions these keywords: date dépense dépenses, vicinal de bruxelles, dépense dépenses, anciennement art, dépenses dépense date, date dépense, fer vicinal, chemin de fer, capital supplémentaire...
Bxl_1926_Tome_II1_Part_4.txt mentions these keywords: anciennement art, obligation compagnie belge, obligation compagnie, obligations tramways, chemin de fer, obligations chemin, compagnie belge chemins, francs le kilogramme, budget des hospices...
Bxl_1926_Tome_II1_Part_5.txt mentions these keywords: anciennement art, vicinal de bruxelles, menus travaux d'entretien, chemin de fer, fer vicinal, entretien menus travaux...
Bxl_1927_Tome_I_Part_10.txt mentions these keywords: chemin de fer, francs le kilogramme, frais d'entretien, magasin centra

In [55]:
cluster_9 = ['Bxl_1920_Tome_I2_Part_15.txt',
     'Bxl_1920_Tome_I2_Part_16.txt',
     'Bxl_1920_Tome_I2_Part_17.txt',
     'Bxl_1920_Tome_I2_Part_2.txt',
     'Bxl_1923_Tome_I2_Part_3.txt',
     'Bxl_1925_Tome_II1_1_Part_3.txt',
     'Bxl_1925_Tome_II1_1_Part_4.txt',
     'Bxl_1926_Tome_I2_Part_16.txt',
     'Bxl_1926_Tome_I2_Part_6.txt',
     'Bxl_1926_Tome_II1_Part_16.txt',
     'Bxl_1926_Tome_II1_Part_6.txt',
     'Bxl_1928_Tome_II1_Part_2.txt',
     'Bxl_1928_Tome_II2_Part_7.txt',
     'Bxl_1928_Tome_II2_Part_8.txt',
     'Bxl_1928_Tome_II2_Part_9.txt',
     'Bxl_1929_Tome_II1_Part_3.txt',
     'Bxl_1929_Tome_II1_Part_4.txt']

In [56]:
for f in sorted(cluster_9):
    text = open(f'{data_path}/{f}', encoding='utf-8').read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for score, kw in keywords:
        words = kw.split()
        if len(words) > 1 and kw not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

Bxl_1920_Tome_I2_Part_15.txt mentions these keywords: entretien des toitures, travaux de jardinage, agents spéciaux, prévisions évaluations recettes...
Bxl_1920_Tome_I2_Part_16.txt mentions these keywords: voir bulletin communal, bulletin communal, voir bulletin, frais d'administration...
Bxl_1920_Tome_I2_Part_17.txt mentions these keywords: maison rue, voir bulletin communal, recettes effectuées, bulletin communal, cahier d'explications, voir bulletin...
Bxl_1920_Tome_I2_Part_2.txt mentions these keywords: date dépense dépenses, dépense dépenses, service ordinaire, l'école professionnelle, service extraordinaire, reçu recettes, nature des recettes, recettes effectuées...
Bxl_1923_Tome_I2_Part_3.txt mentions these keywords: vicinal de bruxelles, fer vicinal, date dépense dépenses, bruxelles à ninove, bruxelles à humbeek, chemin de fer, bruxelles à haecht, dépenses date dépense, commune de laeken, reçu recettes...
Bxl_1925_Tome_II1_1_Part_3.txt mentions these keywords: vicinal de bruxel